# Goodinfo! 還原股價股價與資料處理

In [30]:
def Ex_dividend(stockNo,Name):
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    import time
    from datetime import datetime
    import random
    starttime=datetime.now()#紀錄爬取該支股票要多久

    url='https://goodinfo.tw/StockInfo/StockDividendSchedule.asp?STOCK_ID='+stockNo

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
    }
    res = requests.get(url, headers = headers)
    res.encoding ='utf-8'
    soup = BeautifulSoup(res.text, 'lxml') 

    data = soup.select_one('div#divDetail')
    # print(res.text)#確認一下
    dfs = pd.read_html(data.prettify())
    df = dfs[0]
    df.insert(0,column="有價證券代號",value=stockNo)
    df.insert(1,column="名稱",value=Name)
    df.columns=["有價證券代號","名稱",'股利發放年度','股利所屬盈餘期間','股東會日期','除息交易日','除息參考價','填息完成日','填息花費日數','現金股利發放日','除權交易日','除權參考價','填權完成日','填權花費日數','現金股利盈餘','現金股利公積','現金股利合計','股票股利盈餘','股票股利公積','股票股利合計','股利合計']
    # time.sleep (random.randrange(1, 16))# 據網路上爬取過的使用者分享，爬取間隔 15 sec 比較不容易被擋，之後大量爬取時再解開這行註解。

    # mask = df['除息交易日'].isnull() != True #除息交易日也有空值的時候

    mask = df['除息參考價'].isnull() != True
    df2 = df[mask].reset_index(drop=True)
    df2['除息交易日']=df2['除息交易日'].str.replace('即將除息','').str.replace("/","").str.replace(" ","")

    df2['除息交易日']= df2['除息交易日'].astype(str).str.split("'").str.get(1).astype(str)
    df2['股利發放年度']=df2['股利發放年度'].astype(str)
    df2['除息交易日']=pd.to_datetime((df2['股利發放年度']+df2['除息交易日']).astype(str))
    df2=df2[['有價證券代號','名稱','除息交易日','除息參考價']]
    df2.columns=['stock_id','stock_name','date','reference_price']
    #中文對照：股票代碼/公司名稱/除息交易日/除息參考價

    file_name = "{}_Ex_dividend.csv".format(stockNo)
    df2.to_csv(file_name, index=False)
    print(file_name+"下載完成...", end="")
    endtime= datetime.now()#紀錄爬取該支股票要多久
    print("執行時間:",endtime-starttime,"秒")#紀錄爬取該支股票要多久

In [31]:
import csv
with open('stock_list.csv') as f:
    list_of_stock = csv.DictReader(f)
    for obj in list_of_stock:
        Ex_dividend(obj['stock_id'],obj['stock_name'])

1101_Ex_dividend.csv下載完成...執行時間: 0:00:00.960567 秒
1102_Ex_dividend.csv下載完成...執行時間: 0:00:00.847331 秒
1103_Ex_dividend.csv下載完成...執行時間: 0:00:00.747139 秒


In [32]:
import pandas as pd
check = pd.read_csv('1101_Ex_dividend.csv',sep=',')

In [33]:
check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   stock_id         26 non-null     int64  
 1   stock_name       26 non-null     object 
 2   date             26 non-null     object 
 3   reference_price  26 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 960.0+ bytes


In [34]:
check.head()

,stock_id,stock_name,date,reference_price
0,1101,台泥,2020-08-12,41.90
1,1101,台泥,2019-08-13,38.00
2,1101,台泥,2018-07-26,39.65
3,1101,台泥,2017-07-25,34.55
4,1101,台泥,2016-07-25,33.50
